In [1]:
import copy
import cv2 as cv
import numpy as np

In [2]:
def load_image_into_grayscale(img_path):
    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
    return img

In [3]:
def match_template(template, img):
    # Getting template dimensions
    h, w = template.shape
    
    # Applying template matching
    res = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
    
    # Getting the max similarity score of our template matching
    _, max_val, _, _ = cv.minMaxLoc(res)
    
    # Returning the similarity score
    return max_val

In [4]:
def align_images(reference_img, given_img):
    # Initializing the SIFT algorithm
    sift = cv.SIFT_create()
    
    # Detecting keypoints and descriptors
    keypoints_ref, descriptors_ref = sift.detectAndCompute(reference_img, None)
    keypoints_img, descriptors_img = sift.detectAndCompute(given_img, None)
    
    # Creating the matcher object
    matcher = cv.DescriptorMatcher.create(cv.DescriptorMatcher_BRUTEFORCE)
    
    # Matching the descriptors
    matches = matcher.knnMatch(descriptors_ref, descriptors_img, k=2)
    
    # Applying the ratio test on our matches
    chosen_matches, lowe_ratio = [], 0.75
    for m in matches:
        if m[0].distance / m[1].distance < lowe_ratio:
            chosen_matches.append(m[0])
    
    # Extracting the matched keypoints
    matched_keypoints_ref = np.float32([keypoints_ref[m.queryIdx].pt for m in chosen_matches])
    matched_keypoints_img = np.float32([keypoints_img[m.trainIdx].pt for m in chosen_matches])
    
    # Getting the transformation matrix
    homography_matrix, _ = cv.findHomography(matched_keypoints_img, matched_keypoints_ref, cv.RANSAC)
    
    # Applying the transformation matrix to align the given image with the reference image
    aligned_img = cv.warpPerspective(given_img, homography_matrix, (reference_img.shape[1], reference_img.shape[0]))
    
    return aligned_img

In [5]:
def load_templates(path_tokens, path_tiles):
    templates, template_names = [], ['Token 0', 'Token 1', 'Token 2', 'Token 3', 'Token 4', 'Token 5', 'Token 6', 'Token 7', 'Token 8', 'Token 9', 'Token 10', 'Token 11', 'Token 12', 'Token 13', 'Token 14', 'Token 15', 'Token 16', 'Token 17', 'Token 18', 'Token 19', 'Token 20', 'Token 21', 'Token 24', 'Token 25', 'Token 27', 'Token 28', 'Token 30', 'Token 32', 'Token 35', 'Token 36', 'Token 40', 'Token 42', 'Token 45', 'Token 48', 'Token 49', 'Token 50', 'Token 54', 'Token 56', 'Token 60', 'Token 63', 'Token 64', 'Token 70', 'Token 72', 'Token 80', 'Token 81', 'Token 90', 'Tile blank', 'Tile 2x 1', 'Tile 2x 2', 'Tile 2x 3', 'Tile 2x 4', 'Tile 3x 1', 'Tile 3x 2', 'Tile 3x 3', 'Tile 3x 4', 'Tile 3x 5', 'Tile 3x 6', 'Tile 3x 7', 'Tile 3x 8', 'Tile +', 'Tile - 1', 'Tile - 2', 'Tile x', 'Tile : 1', 'Tile : 2', 'Tile 1', 'Tile 2', 'Tile 3', 'Tile 4']
    
    tokens_count, count = 46, 0
    imgs_per_line, imgs_per_col = 8, 6
    
    for i in range(imgs_per_col):
        for j in range(imgs_per_line):
            img = load_image_into_grayscale(f'{path_tokens}/{i}_{j}.jpg')
            templates.append(img)
            count += 1
            if count == tokens_count:
                break
        if count == tokens_count:
            break
    
    tiles_count, count = 23, 0
    imgs_per_line, imgs_per_col = 8, 3
    
    for i in range(imgs_per_col):
        for j in range(imgs_per_line):
            img = load_image_into_grayscale(f'{path_tiles}/{i}_{j}.jpg')
            templates.append(img)
            count += 1
            if count == tiles_count:
                break
        if count == tiles_count:
            break
    
    return templates, template_names

In [6]:
def perform_and_check_operation_nearby_two_tiles(tile_line, tile_col, token_placed_value, special_tiles_coords, first_tile_value, second_tile_value):
    plus_sign_tiles_coords = special_tiles_coords[2]
    minus_sign_tiles_coords = special_tiles_coords[3]
    times_sign_tiles_coords = special_tiles_coords[4]
    division_sign_tiles_coords = special_tiles_coords[5]
    
    operation_score = 0
    
    if (tile_line, tile_col) in plus_sign_tiles_coords:
        if token_placed_value == first_tile_value + second_tile_value:
            operation_score = first_tile_value + second_tile_value
    elif (tile_line, tile_col) in minus_sign_tiles_coords:
        if token_placed_value == abs(first_tile_value - second_tile_value):
            operation_score = abs(first_tile_value - second_tile_value)
    elif (tile_line, tile_col) in times_sign_tiles_coords:
        if token_placed_value == first_tile_value * second_tile_value:
            operation_score = first_tile_value * second_tile_value
    elif (tile_line, tile_col) in division_sign_tiles_coords:
        if second_tile_value != 0 and token_placed_value == first_tile_value / second_tile_value:
            operation_score = first_tile_value / second_tile_value
        elif first_tile_value != 0 and token_placed_value == second_tile_value / first_tile_value:
            operation_score = second_tile_value / first_tile_value
    else:
        if token_placed_value == first_tile_value + second_tile_value:
            operation_score = first_tile_value + second_tile_value
        elif token_placed_value == abs(first_tile_value - second_tile_value):
            operation_score = abs(first_tile_value - second_tile_value)
        elif token_placed_value == first_tile_value * second_tile_value:
            operation_score = first_tile_value * second_tile_value
        elif second_tile_value != 0 and token_placed_value == first_tile_value / second_tile_value:
            operation_score = first_tile_value / second_tile_value
        elif first_tile_value != 0 and token_placed_value == second_tile_value / first_tile_value:
            operation_score = second_tile_value / first_tile_value
    
    return operation_score == token_placed_value

In [7]:
def compute_move_score(tile_line, tile_col, token_placed_value, current_move_grid, special_tiles_coords, unoccupied_tile_types):
    times_two_tiles_coords = special_tiles_coords[0]
    times_three_tiles_coords = special_tiles_coords[1]
    
    completed_equations = 0
    if tile_line >= 2: # Checking if there is enough space to the top of the tile for another equation
        first_tile_content = current_move_grid[tile_line - 1][tile_col]
        second_tile_content = current_move_grid[tile_line - 2][tile_col]
        if first_tile_content not in unoccupied_tile_types and second_tile_content not in unoccupied_tile_types:
            if perform_and_check_operation_nearby_two_tiles(tile_line, tile_col, token_placed_value, special_tiles_coords, first_tile_content, second_tile_content):
                completed_equations += 1
    if tile_col <= 11: # Checking if there is enough space to the right of the tile for another equation
        first_tile_content = current_move_grid[tile_line][tile_col + 1]
        second_tile_content = current_move_grid[tile_line][tile_col + 2]
        if first_tile_content not in unoccupied_tile_types and second_tile_content not in unoccupied_tile_types:
            if perform_and_check_operation_nearby_two_tiles(tile_line, tile_col, token_placed_value, special_tiles_coords, first_tile_content, second_tile_content):
                completed_equations += 1
    if tile_line <= 11: # Checking if there is enough space to the bottom of the tile for another equation
        first_tile_content = current_move_grid[tile_line + 1][tile_col]
        second_tile_content = current_move_grid[tile_line + 2][tile_col]
        if first_tile_content not in unoccupied_tile_types and second_tile_content not in unoccupied_tile_types:
            if perform_and_check_operation_nearby_two_tiles(tile_line, tile_col, token_placed_value, special_tiles_coords, first_tile_content, second_tile_content):
                completed_equations += 1
    if tile_col >= 2: # Checking if there is enough space to the left of the tile for another equation
        first_tile_content = current_move_grid[tile_line][tile_col - 1]
        second_tile_content = current_move_grid[tile_line][tile_col - 2]
        if first_tile_content not in unoccupied_tile_types and second_tile_content not in unoccupied_tile_types:
            if perform_and_check_operation_nearby_two_tiles(tile_line, tile_col, token_placed_value, special_tiles_coords, first_tile_content, second_tile_content):
                completed_equations += 1
    
    if completed_equations == 0:
        completed_equations = 1 # (SAFEGUARD) If the algorithm didn't manage to find even one equation, we still know that there should be at least one completed equation
    
    move_score = token_placed_value * completed_equations
    
    if (tile_line, tile_col) in times_two_tiles_coords:
        move_score = move_score * 2
    elif (tile_line, tile_col) in times_three_tiles_coords:
        move_score = move_score * 3
    
    return move_score

In [8]:
def get_newly_occupied_tile_coords(previous_move_grid, current_move_grid, unoccupied_tile_types):
    no_lines, no_cols = 14, 14 # Grid dimensions
    
    different_tiles = []
    for i in range(no_lines):
        for j in range(no_cols):
            if previous_move_grid[i][j] != current_move_grid[i][j]:
                if previous_move_grid[i][j] in unoccupied_tile_types and current_move_grid[i][j] not in unoccupied_tile_types:
                    different_tiles.append((i, j))
    
    if len(different_tiles) > 0: # (SAFEGUARD) If the algorithm mistakenly found more than one tile that has been newly occupied, we take only the first such tile 
        coords = (different_tiles[0][0], different_tiles[0][1])
    else: # (SAFEGUARD) The algorithm was unable to find a newly occupied tile
        coords = (None, None)

    return coords

In [9]:
def get_tile_content(templates, template_names, value_img):
    template_index, max_similarity = 0, -1
    for i in range(len(templates)):
        similarity = match_template(templates[i], value_img)
        if similarity > max_similarity:
            max_similarity = similarity
            template_index = i
    
    # (SAFEGUARD) If the template matching algorithm is indecisive, we treat the tile as blank, 
    # as there are very high chances of actually being a blank tile, given the dimensions and structure of the templates
    if max_similarity < 0.7:
        tile_content = 'blank'
    else:
        if template_names[template_index].split()[0] == 'Token' or (template_names[template_index].split()[0] == 'Tile' and template_names[template_index].split()[1] in ['1', '2', '3', '4']):
            tile_content = int(template_names[template_index].split()[1])
        else:
            tile_content = template_names[template_index].split()[1]
    
    return tile_content

In [10]:
def get_special_tiles_coords():
    times_two_tiles_coords = [(1, 1), (2, 2), (3, 3), (4, 4), (4, 9), (3, 10), (2, 11), (1, 12), (12, 1), (11, 2), (10, 3), (9, 4), (9, 9), (10, 10), (11, 11), (12, 12)]
    times_three_tiles_coords = [(0, 0), (0, 6), (0, 7), (0, 13), (6, 0), (6, 13), (7, 0), (7, 13), (13, 0), (13, 6), (13, 7), (13, 13)]
    plus_sign_tiles_coords = [(3, 6), (4, 7), (6, 4), (7, 3), (6, 10), (7, 9), (9, 6), (10, 7)]
    minus_sign_tiles_coords = [(2, 5), (2, 8), (5, 2), (8, 2), (5, 11), (8, 11), (11, 5), (11, 8)]
    times_sign_tiles_coords = [(3, 7), (4, 6), (6, 3), (7, 4), (6, 9), (7, 10), (9, 7), (10, 6)]
    division_sign_tiles_coords = [(1, 4), (1, 9), (4, 1), (9, 1), (4, 12), (9, 12), (12, 4), (12, 9)]
    special_tiles_coords = (times_two_tiles_coords, times_three_tiles_coords, plus_sign_tiles_coords, minus_sign_tiles_coords, times_sign_tiles_coords, division_sign_tiles_coords)
    return special_tiles_coords

In [11]:
def get_grid(templates, template_names, aligned_img, start_x=76, start_y=76, extra_tile_coverage=0):
    # start_x, start_y: roughly the coordinates in the aligned image of the center of the first tile of the grid (from upper left corner)
    # extra_tile_coverage: how many extra columns/rows of pixels to add to the left/right/top/bottom of each tile image
    
    no_lines, no_cols = 14, 14 # Grid dimensions
    tile_dim, tile_half_dim = 152, 76 # We treat each tile as a square, so only one dimension is needed
    
    # We first crop each tile from the aligned image
    grid_imgs = []
    for line in range(no_lines):
        grid_line = []
        for col in range(no_cols):
            # We first get the x and y coordinates in the aligned image of the center of the current tile
            tile_center_x, tile_center_y = start_x + (tile_dim * col), start_y + (tile_dim * line)
            # How many pixels the tile will have to its left/right/top/bottom
            tile_space = tile_half_dim + extra_tile_coverage
            # Cropping the tile from the aligned image
            tile_img = aligned_img[tile_center_y - tile_space:tile_center_y + tile_space, tile_center_x - tile_space:tile_center_x + tile_space]
            
            grid_line.append(tile_img)
        grid_imgs.append(grid_line)
    
    # We then extract the content from each tile image
    grid_content = []
    for line in grid_imgs:
        grid_line = []
        for tile_img in line:
            tile_content = get_tile_content(templates, template_names, tile_img)
            grid_line.append(tile_content)
        grid_content.append(grid_line)
    
    return grid_content

In [12]:
def compute_game_results(game_no, input_turns_file_path, input_moves_imgs_path, output_moves_txts_path, output_scores_file_path, templates, template_names):
    turns = []
    with open(input_turns_file_path) as file:
        lines = file.readlines()
        for line in lines:
            line = line.rstrip()
            turn = line.split()
            turn = (turn[0], int(turn[1]))
            turns.append(turn)
    
    special_tiles_coords = get_special_tiles_coords()
    
    unoccupied_tile_types = ['blank', '2x', '3x', '+', '-', 'x', ':']
    
    # For writing to file
    grid_col_no_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N'}
    
    reference_board_alignment_img_path = 'templates/boards/reference_alignment.jpg'
    reference_board_alignment_img = load_image_into_grayscale(reference_board_alignment_img_path)
    
    start_game_grid_img_path = 'templates/boards/start.jpg'
    start_game_grid_img = load_image_into_grayscale(start_game_grid_img_path)
    previous_move_grid = get_grid(templates, template_names, start_game_grid_img)
    
    scores_file = open(output_scores_file_path, 'w')
    
    for turn in range(len(turns)):
        # We prepare the info for the current turn (player name, turn start, turn end)
        player, turn_start = turns[turn][0], turns[turn][1]
        if turn <= len(turns) - 2:
            turn_end = turns[turn + 1][1]
        else:
            turn_end = 51
        
        turn_score = 0
        
        # We then compute and process each turn's move
        for move in range(turn_start, turn_end):
            if move <= 9:
                zero = '0'
            else:
                zero = ''
            move_str = zero + str(move)
            move_img_path = f'{input_moves_imgs_path}/{game_no}_{move_str}.jpg'
            move_img = load_image_into_grayscale(move_img_path)
            
            aligned_board_img = align_images(reference_board_alignment_img, move_img)
            
            first_tile_center_x, first_tile_center_y = 421, 431 # Hard coded coordinates in the aligned board image of the center of the upper left corner grid tile
            extra_tile_coverage = 10 # Check the function "get_grid" for the meaning of this variable
            current_move_grid = get_grid(templates, template_names, aligned_board_img, first_tile_center_x, first_tile_center_y, extra_tile_coverage)
            
            coords = get_newly_occupied_tile_coords(previous_move_grid, current_move_grid, unoccupied_tile_types)
            tile_line, tile_col = coords[0], coords[1]
            
            if tile_line is not None:
                token_placed_value = current_move_grid[tile_line][tile_col]
                move_score = compute_move_score(tile_line, tile_col, token_placed_value, current_move_grid, special_tiles_coords, unoccupied_tile_types)
            else: # (SAFEGUARD) In case the algorithm is unable to compute the score of the move
                move_score = 0
            with open(f'{output_moves_txts_path}/{game_no}_{move_str}.txt', 'w') as file:
                file.write(str(tile_line + 1) + grid_col_no_to_letter[tile_col] + ' ' + str(token_placed_value))
            
            turn_score += move_score
            
            previous_move_grid = copy.deepcopy(current_move_grid)
        
        if turn_end == 51:
            nl = ''
        else:
            nl = '\n'
        scores_file.write(player + ' ' + str(turn_start) + ' ' + str(turn_score) + nl)
    
    scores_file.close()

In [13]:
path_template_tokens, path_template_tiles = 'templates/tokens/', 'templates/tiles/'
templates, template_names = load_templates(path_template_tokens, path_template_tiles)

In [14]:
# Insert game number
game_no = 1
# Insert path to game turns .txt file
input_turns_file_path = 'train/1_turns.txt'
# Insert path to the folder containing the game moves images
input_moves_imgs_path = 'train/'
# Insert path to the folder that will contain the game moves .txt files
output_moves_txts_path = 'train_res/'
# Insert path to game scores .txt file
output_scores_file_path = 'train_res/res_train_1_scores.txt'

In [15]:
compute_game_results(game_no, input_turns_file_path, input_moves_imgs_path, output_moves_txts_path, output_scores_file_path, templates, template_names)